In [ ]:
# permisos para acceder a googleDrive
from google.colab import drive
drive.mount('/content/drive')

In [26]:
import os

drive_folder = 'drive/MyDrive/TESIS/'

# Definición de la base de datos
data_folder = drive_folder + 'db/data/'
image_folder = drive_folder + 'db/images/cow/'
shape_folder = drive_folder + 'db/images/shape/'

# si no da error es que se cargaron bien
files = os.listdir(image_folder)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import os
import torch
import torchvision
import torchvision.transforms as T
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Modelo preentrenado de DeepLabV3
model = torchvision.models.segmentation.deeplabv3_resnet101(weights=torchvision.models.segmentation.DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1)
model.eval()

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [28]:
# Transformación inversa para redimensionar la máscara a las dimensiones originales de la imagen
def resize_mask(mask, original_size):
    mask = Image.fromarray(mask.byte().cpu().numpy()).resize(original_size, resample=Image.NEAREST)
    return np.array(mask)

# Función para segmentar la imagen
def segment_image(image):
    input_image = transform(image).unsqueeze(0)
    with torch.no_grad():
        output = model(input_image)['out'][0]
    output_predictions = output.argmax(0)
    return output_predictions

# Función para aplicar la máscara segmentada y extraer la vaca
def extract_cow(image, mask):
    original_size = image.size
    mask = resize_mask(mask, original_size)
    image_np = np.array(image)
    extracted_image = image_np * np.expand_dims(mask, axis=2)
    return Image.fromarray(extracted_image)

In [ ]:
# Procesar cada imagen en el directorio
for image_name in os.listdir(image_folder):
    if image_name.endswith('.jpg') or image_name.endswith('.png'):
        image_path = os.path.join(image_folder, image_name)
        image = Image.open(image_path).convert("RGB")

        # Segmentar la imagen
        mask = segment_image(image)

        # Extraer la vaca
        extracted_image = extract_cow(image, mask)

        # Mostrar el resultado
        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
        ax[0].imshow(image)
        ax[0].set_title('Original Image')
        ax[0].axis('off')

        ax[1].imshow(extracted_image)
        ax[1].set_title('Extracted Cow')
        ax[1].axis('off')

        plt.show()